# Life Expectancy Prediction
Importing Dataset to IBM watson Studio


In [3]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

#@hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e08f5880c4f7463fba124cab9de66159 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="",
    config=Config(signature_version='oauth'),
    endpoint_url='')

body = client_e08f5880c4f7463fba124cab9de66159.get_object(Bucket='lifeexpectancyrandomforest-donotdelete-pr-zc9us8qxthugdk',Key='Life Expectancy Data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
data = pd.read_csv(body)
data.head()



ModuleNotFoundError: No module named 'botocore'

In [ ]:
#DATA PREPROCESSING
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
country_list = data.Country.unique()
len(country_list)

In [ ]:
country_list = data.Country.unique()
fill_list = ['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling']

In [7]:
for country in country_list:
    data.loc[data['Country'] == country,fill_list] = data.loc[data['Country'] == country,fill_list].interpolate()
data.dropna(inplace=True)

In [8]:
data.isna().sum()

Country                            0
Year                               0
Status                             0
Life expectancy                    0
Adult Mortality                    0
infant deaths                      0
Alcohol                            0
percentage expenditure             0
Hepatitis B                        0
Measles                            0
 BMI                               0
under-five deaths                  0
Polio                              0
Total expenditure                  0
Diphtheria                         0
 HIV/AIDS                          0
GDP                                0
Population                         0
 thinness  1-19 years              0
 thinness 5-9 years                0
Income composition of resources    0
Schooling                          0
dtype: int64

# Renaming Columns

In [9]:
data.rename(columns={" BMI ":"BMI",'Life expectancy ':'Life expectancy',
                  "under-five deaths ":"under-five deaths","Measles ":"Measles","Diphtheria ":"Diphtheria",
                  ' HIV/AIDS':"HIV/AIDS",
                  " thinness  1-19 years":"thinness 10-19 years"," thinness 5-9 years":"thinness 5-9 years"},inplace=True)

In [16]:
X = data.drop('Life_Expectancy', axis=1)
Y = pd.DataFrame(data=data,columns=['Life_Expectancy'])

In [18]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from collections import OrderedDict
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error

In [19]:
#Training and testing Data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [20]:
numeric_features = ['Year', 'BMI',
       'Adult_Mortality', 'Infant_Deaths', 'Alcohol', 'Percentage_Expenditure',
       'Hepatitis_B', 'Under_Five_Deaths', 'Polio', 'Total_Expenditure',
       'Diphtheria', 'HIV/AIDS', 'GDP', 'Population', 'Thinness_10_19_years',
       'Thinness_5_9_years', 'Income_Composition_of_Resources', 'Schooling',
       'Measles']
categorical_features = ['Country', 'Status']

In [21]:
#Encoding Categorical Data
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

In [22]:

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
   
])

In [23]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
    ]
)

In [24]:
RFmodel = Pipeline([
     ('preprocessor', preprocessor),
     ('RFmodel', RandomForestRegressor())
])

In [25]:
RFmodel.fit(X_train,Y_train)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('cat', Pipeline(memory=None,
     steps=[('onehot', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>...ators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [26]:
predict= RFmodel.predict(X_test)

In [27]:
r2_score(predict, Y_test)

0.9539673596450048

In [28]:
!pip install watson-machine-learning-client

In [29]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-06-03 08:08:14,897 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [30]:
wml_credentials={
 "apikey":################,
  "iam_apikey_description": #######################,
  "iam_apikey_name":  #######################,
  "iam_role_crn":  #######################,
  "iam_serviceid_crn":  #######################,
  "instance_id":  #######################,
  "url": #######################
}

In [31]:
client = WatsonMachineLearningAPIClient( wml_credentials )

In [32]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Dhanush Amin", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "dhanushamin10@gmail.com", 
               client.repository.ModelMetaNames.NAME: "LifeExpectancy"}

In [33]:
model_artifact =client.repository.store_model(RFmodel, meta_props=model_props)

In [34]:
published_model_uid = client.repository.get_model_uid(model_artifact)
published_model_uid

'9a1bdeb7-a5dd-4950-9b4d-5e63f533f512'

In [35]:
deployment = client.deployments.create(published_model_uid, name="life_expectancy")
scoring_endpoint = client.deployments.get_scoring_url(deployment)
scoring_endpoint



#######################################################################################

Synchronous deployment creation for uid: '9a1bdeb7-a5dd-4950-9b4d-5e63f533f512' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='e097cf54-e144-48e0-b6bc-50320bb7fa52'
------------------------------------------------------------------------------------------------




'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/8aa1bd59-fd42-450b-9215-86d97ac93f45/deployments/e097cf54-e144-48e0-b6bc-50320bb7fa52/online'